<a href="https://colab.research.google.com/github/insight4healthlab/course-GS-HLTH-6270/blob/main/notebooks/Demo_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NER Demo

1)
Make a copy of this notebook
(file -> Save a copy in Drive)

2)
Go to https://huggingface.co/ and create a account.

Go to your profile -> Settings -> Access Tokens -> Create new token

Select "Read" as the type of token and make sure to save it for later reference.

In [ ]:
#Import necessary packages and libraries
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from huggingface_hub import login
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy
import seaborn as sns

In [ ]:
HF_Token= 'Your_Token'
login(HF_Token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#Disease Identification; Two Classes: Disease and O (Non-Disease)

---



In [ ]:
#Sample text produced by GPT-4
text="Diseases affect various organs and systems in the human body. One common illness is influenza, or the flu, a viral infection that targets the respiratory system. It spreads easily, leading to symptoms such as fever, cough, sore throat, and fatigue. Severe cases can result in pneumonia. Vaccinations help prevent the disease. Diabetes is a chronic condition characterized by elevated blood sugar levels. In Type 1 diabetes, the pancreas produces little to no insulin. In Type 2, the body becomes resistant to insulin. If not managed, diabetes can cause heart disease, kidney failure, and nerve damage. Treatment includes lifestyle changes, medication, and insulin injections. Hypertension, or high blood pressure, is another condition that can have severe health consequences. Often called the 'silent killer', it increases the risk of heart attack, stroke, and kidney disease. Regular monitoring and medication help control it. Cancer is a group of diseases involving abnormal cell growth. Breast cancer and lung cancer are among the most common forms. Treatment depends on the type and stage of cancer, with options including surgery and chemotherapy. Tuberculosis (TB) is an infectious disease caused by bacteria, primarily affecting the lungs. Symptoms include a persistent cough and fever. TB is contagious but treatable with antibiotics."

In [ ]:
#Import NER Model and Tokenizer
model= AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
tokenizer= AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#Define the pipeline
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="max")

In [ ]:
#Apply pipeline on the text
ner_results=nlp_ner(text)

In [ ]:
#Check the output
ner_results

[{'entity_group': '0',
  'score': 0.9988529,
  'word': 'Diseases affect various organs and systems in the human body. One common illness is',
  'start': 0,
  'end': 83},
 {'entity_group': 'DISEASE',
  'score': 0.9999907,
  'word': 'influenza',
  'start': 84,
  'end': 93},
 {'entity_group': '0',
  'score': 0.9999965,
  'word': ', or the',
  'start': 93,
  'end': 101},
 {'entity_group': 'DISEASE',
  'score': 0.9992235,
  'word': 'flu',
  'start': 102,
  'end': 105},
 {'entity_group': '0',
  'score': 0.99999887,
  'word': ', a',
  'start': 105,
  'end': 108},
 {'entity_group': 'DISEASE',
  'score': 0.9999871,
  'word': 'viral infection',
  'start': 109,
  'end': 124},
 {'entity_group': '0',
  'score': 0.9999987,
  'word': 'that targets the respiratory system. It spreads easily, leading to symptoms such as',
  'start': 125,
  'end': 208},
 {'entity_group': 'DISEASE',
  'score': 0.9999856,
  'word': 'fever',
  'start': 209,
  'end': 214},
 {'entity_group': '0',
  'score': 0.9999988,
  'word

In [ ]:
#Visualize the results
nlp_visualize= spacy.blank("en")
document = nlp_visualize(text)
entities = []
for entity in ner_results:
    start = entity['start']
    end = entity['end']
    label = entity['entity_group']

    span = document.char_span(start, end, label=label, alignment_mode="contract")

    if span is not None:
        entities.append(span)

# Set the entities on the doc
document.ents = entities
colors = {
    "DISEASE": "#f5a623"
}

# Define visualization options
options = {
    "colors": colors
}

# Visualize the text with entities using displacy
displacy.render(document, style="ent", options=options,jupyter=True)

#Multi-Label Medical NER in CoNLL format



In [ ]:
#Use same text as before
text=text

In [ ]:
#Import NER Model and Tokenizer
model= AutoModelForTokenClassification.from_pretrained("medical-ner-proj/bert-medical-ner-proj")
tokenizer= AutoTokenizer.from_pretrained("medical-ner-proj/bert-medical-ner-proj")

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
#Define the pipeline
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="max")

In [ ]:
#Apply pipeline on the text
ner_results=nlp_ner(text)

In [ ]:
#Check the output
ner_results

[{'entity_group': 'B_problem',
  'score': 0.98549545,
  'word': 'diseases',
  'start': 0,
  'end': 8},
 {'entity_group': 'B_problem',
  'score': 0.996262,
  'word': 'influenza',
  'start': 84,
  'end': 93},
 {'entity_group': 'B_problem',
  'score': 0.99398005,
  'word': 'the',
  'start': 98,
  'end': 101},
 {'entity_group': 'I_problem',
  'score': 0.9934784,
  'word': 'flu',
  'start': 102,
  'end': 105},
 {'entity_group': 'B_problem',
  'score': 0.990008,
  'word': 'a',
  'start': 107,
  'end': 108},
 {'entity_group': 'I_problem',
  'score': 0.9961981,
  'word': 'viral infection',
  'start': 109,
  'end': 124},
 {'entity_group': 'B_pronoun',
  'score': 0.99470043,
  'word': 'that',
  'start': 125,
  'end': 129},
 {'entity_group': 'B_pronoun',
  'score': 0.99275887,
  'word': 'it',
  'start': 162,
  'end': 164},
 {'entity_group': 'B_problem',
  'score': 0.9747971,
  'word': 'symptoms',
  'start': 192,
  'end': 200},
 {'entity_group': 'B_problem',
  'score': 0.9973966,
  'word': 'fever'

In [ ]:
#Visualize the results
nlp_visualize= spacy.blank("en")
document = nlp_visualize(text)
entities = []
for entity in ner_results:
    start = entity['start']
    end = entity['end']
    label = entity['entity_group']

    span = document.char_span(start, end, label=label, alignment_mode="contract")

    if span is not None:
        entities.append(span)

# Set the entities on the doc
document.ents = entities

#Determine color settings
unique_labels = set(entity['entity_group'] for entity in ner_results)
palette = sns.color_palette("pastel", len(unique_labels))
colors = {label: f'#{int(color[0]*255):02x}{int(color[1]*255):02x}{int(color[2]*255):02x}' for label, color in zip(unique_labels, palette)}

# Define visualization options
options = {
    "colors": colors
}

# Visualize the text with entities using displacy
displacy.render(document, style="ent", options=options,jupyter=True)